In [1]:
from numpy.random import randint
import numpy as np
from qiskit import *
from qiskit import IBMQ
from qiskit.visualization import plot_histogram,plot_bloch_multivector
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
from math import pi
import math

In [2]:
n=100
np.random.seed()
alice_bits=list(randint(2,size=n))
print(alice_bits)
print("first",alice_bits[0])

[1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1]
first 1


In [3]:
def encode_message(a,b):
    n=len(a)
    message=[]
    #q=QuantumRegister(n)
    #c=ClassicalRegister(n)
    for i in range(n):
        qc=QuantumCircuit(1,1)
        if b[i]==0:#computational
            if a[i]==0:
                pass
            else:
                qc.x(0)
        else:
            if a[i]==0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        message.append(qc)
    return message


In [4]:
alice_bases=list(randint(2,size=n))
message=encode_message(alice_bits,alice_bases)
print(alice_bases)
#message.draw()
#print(message[101])

[1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0]


In [5]:
def intercept_message(m,b):
    backend=Aer.get_backend("qasm_simulator")
    res=[]
    n=len(b)
    for i in range(n):
        if b[i]==0:
            message[i].measure(0,0)
        else:
            message[i].h(0)
            message[i].measure(0,0)
            message[i].h(0)
        result=execute(message[i],backend,shots=1,memory=True).result()
        m_bit=int(result.get_memory()[0])
        res.append(m_bit)
    return res

In [6]:
def intercept_message2(m,b):
    backend=Aer.get_backend("qasm_simulator")
    res=[]
    n=len(b)
    for i in range(n):
        message[i].ry(-math.pi/4,0)
        message[i].measure(0,0)
        message[i].ry(math.pi/4,0)
        result=execute(message[i],backend,shots=1,memory=True).result()
        m_bit=int(result.get_memory()[0])
        res.append(m_bit)
    return res

In [7]:
#(a)
eve_bases=list(randint(2,size=n))
intercepted_message=intercept_message(message,eve_bases)


In [7]:
#(b)
eve_bases=list(randint(2,size=n))
intercepted_message=intercept_message2(message,eve_bases)

In [8]:
bob_bases=list(randint(2,size=n))
bob_bits=intercept_message(message,bob_bases)
print(bob_bits)

[1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1]


In [9]:
def remove_garbage(a_base,b_base,bits):
    lis=[]
    for i in range(len(a_base)):
        if a_base[i]!=b_base[i]:
            lis.append(i)
    return np.delete(bits,lis)

In [10]:
alice_key=remove_garbage(alice_bases,bob_bases,alice_bits)
bob_key=remove_garbage(alice_bases,bob_bases,bob_bits)

In [11]:
print(alice_key)
print(bob_key)
print(len(alice_key))

[1 1 0 1 0 1 0 0 1 1 1 0 0 1 0 0 1 1 1 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 0 1 1
 0 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1]
[1 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 1 0 1 1 1 0 0 1 1 0 0 1 1 0 0 0 1 1 0 1 0
 1 1 1 1 0 0 0 1 1 1 1 0 1 1 0 1]
53


In [12]:
count=0
for j in range(len(alice_key)):
    if alice_key[j]==bob_key[j]:
        count+=1
print(count)

39


In [13]:
c=count/len(alice_key)
print(c)

0.7358490566037735
